IMPORTANT NOTE: This notebook is for learning and entertainment purposes only. Please consult robot.txt files before scraping

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Note: This class needs some serious rewriting before general usage. 
# For the purposes of creating this corpus, it'll do...
class web_crawl():
	def __init__(self, allowed_domains: list[str]):
		
		# Set up chrome driver with options
		options = Options()
		options.add_argument("--headless")
		options.add_argument("--disable-gpu")
		driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
								options=options)
		
		# Initialize variables
		self.driver = driver
		self.links = []
		self.directories = []
		self.result = []
		self.allowed_domains = allowed_domains
	
	def get_soup(self, link: str):

		# Navigate to URL
		self.driver.get(link)

		# Retrieve the page source
		html = self.driver.page_source

		# Parse the HTML with BeautifulSoup
		return BeautifulSoup(html, 'html.parser')

	def web_scrape(self, link: str): # User: Make sure link is in list of allowed domains
		
		# initialize variables
		text = ''
		links = []
		soup = self.get_soup(link)
		
		# let's assume all the text is wrapped around <p> tags.
		for element in soup.find_all('p'):
			text += (element.text) + ' '
		
		# let's assume some of the text is wrapped around <span> tags.
		for element in soup.find_all('span'):
			text += (element.text) + ' '

		# let's assume some of the text is wrapped around <a> tags.
		for element in soup.find_all('a'):
			text += (element.text) + ' '

		# let's assume all the links are in blocks with href.
		for element in soup.find_all('a', href=True):
			href = element['href']
			if (self.check_domain(href)) and (href not in self.links):
				links.append(href)
			if (href.startswith('/')) and (href not in self.directories):
				links.append(link + href)
				self.directories.append(href)

		# Output ordered pair
		return text, links
		
	def dfs(self, link: str): # User: Make sure link is in list of allowed domains
		
		# Web scrape
		text, links = self.web_scrape(link=link)
		if (link, text) not in self.result: # Check unique strings
			self.result.append({link : text})
			print(f"Scraped {link}")
		
		# Mark link as explored
		if link not in self.links:
			self.links.append(link)
		
		# Visit neighbors
		for neighbor in links:
			if neighbor not in self.links:
				self.dfs(neighbor)

	def check_link(self, link: str):
		
		# Check for unallowed links
		unallowed_links = ['https://.com', 'https://.org']
		if (link in unallowed_links):
			return False 
		
		# Check for mailing links
		if ("mailto:" in link):
			return False
		
		# Check if it's a link
		return ("https://" in link and (".com" in link or ".org" in link))

	def check_domain(self, link: str):
		
		# Check if it's even a link
		if (not self.check_link(link)):
			return False
		
		# Get domain
		token = link.split('https://')[1].split('/')[0]
		domain = (token.split('.')[-2]+'.' + token.split('.')[-1]).split('.')[0]
		
		# Check for allowed domains
		return (domain in self.allowed_domains)

	
	def __del__(self):
		
		# Close the driver
		self.driver.quit()
		